# Caso de uso

### Contexto

Con mi actual equipo de trabajo tenemos la misión de generar una herramienta para la gestión a tiempo real de recursos y avance de operaciones, y el levantamiento de datos obtenidos a través de ensayos no destructivos en paradas generales de planta (PGP). Este desarrollo se está realizando y testeando constantemente al momento de prestar servicios en planes de inspecciones preventivos para lineas de producción en la industría de la celulosa, principalmente Arauco y CMPC. Todo este proceso es realizado a manera de prevenir detenciones de producción hasta la próxima PGP, asegurando su correcto y óptimo operamiento.

Si bien las bases de datos alimentadas en estos procesos no son accesibles, están de momento no son tan significativas dado que el tiempo de desde el incicio de este proyecto no es el suficiente, se planteara un análisis basado en mi experiecia apoyado en psedo código con los lineamientos de este módulo para complementar el servicio ya prestado.

### Aspectos técnicos

El enfoque de este análisis será basado en los planes de inspección preventivos en calderas de poder y calderas recuperadores en líneas de producción de celulosa. Para las cuales una forma simplificada del flujo de trabajo se rige de la siguiente manera:
1. Planificación del programa de inspección y posterior aprobación por planta.
2. Entrega desde la empresa de celulosa del equipo en cuestión.
3. Preparación de andamios.
4. Limpieza superficial de componentes a analizar.
5. Ensayos no destructivos, de los cuales principalmente nos guiaremos en la medición de espesores (TML).
6. Gestión de datos, agregar espesores en la base de datos de manera correcta, verificación de espesores bajo el mínimo según los protocolos estándar para su corroboración.
7. Auditoría de datos basado en la desviación de estándar por componente e histórico.
8. Análisis de vida remanente para componentes críticos.
9. Informe a planta para cambios necesarios.
10. Iteración de revisión de soldaduras en cambios, liberando los trabajos que cumplan condiciones estándar.

## Análisis descriptivo

Para el análisis descriptivo nos enfocaremos en los puntos 6, 7 y 8.

### Gestión de datos

Luego que los operadores han limpiado correctamente los componentes acordados en el plan de inspección, y medido bajo los procedimientos estándar los espesores requeridos, estos son gestionados bajo el programa en desarrollo. En este es posible ingresar los datos levantados por componente, y los valores de referencia (según el material y las cualidades de su uso) para los cuales se levanta la primera alerta, en caso de que los espesores estén bajo el espesor mínimo permitido, se le pide a los supervisores que realicen una nueva medición en el lugar crítico, para verificar antes de informar a planta para que tomen las desiciones necesarias para la extensión de vida útil del componente.

Un ánalisis útil para esta parte del proceso sería la generación gráfica basada en un croquis o imagen referencial del componente en cuestión, en donde sobre este se grafiquen puntualmente la posición de las mediciones bajo una colorimetría adecuada de los puntos según su estado en referencia a los valores de funcionamiento especificados por los fabricantes, para el oportuno y claro transpaso de información.

```text
1 Cargar librerías requeridas (pandas, numpy, matplotlib.pyplot, seaborn)

2 Cargar datos y croquis (pos_x : array, pos_y : array, espesores : array, espesores_ref : array, croquis : imagen)

3 Grafico scatter sobre croquis, con colorímetria acorde a los espesores de referencia

4 Exportar gráfico
```

Aquí es importante resaltar que el gráfico mencionado puede ser generado de la siguiente forma:

```python
import matplotlib.pyplot as plt
import numpy as np

# Cargar el croquis como fondo
image = plt.imread('croquis.jpg')

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(8, 6))

# Mostrar la imagen de fondo
ax.imshow(image, extent=[0, x, 0, y], aspect='auto')  # Ajustar 'extent' al sistema de coordenadas

# Dibujar el scatter plot sobre la imagen
scatter = ax.scatter(pos_x, pos_y, c=espesores, cmap='viridis')

# Añadir una barra de color para representar los valores
cbar = plt.colorbar(scatter, ax=ax, label='Colorimetría')

# Ajustar los ejes al tamaño de la imagen
ax.set_xlim(0, x)
ax.set_ylim(0, y)

plt.show()

```

### Auditoría de datos

El segundo de la gestión de los espesores bajos, es importante ver si las mediciones actuales son coeherentes respecto a las mediciones de eventos anteriores y la tasa de desgaste informada por el fabricante, por lo que auditar los datos en base a la desviación estándar admisible para cada especificación de material y componente es clave.

Un análisis posible para esto es generar un gráfico interactivo que nos permita mostrar los lugares de medición en el componente actual (similar al punto anterior) pero con una colorímetria acorde a su desviación estándar, y que en una ventana que se despliegue desde cada punto sea posible ver el valor del espesor inmediatamente anterior, y la desviación estándar que tuvo en este evento, dando principalmente un enfasis en las mediciones bajo la desviación estándar negativa no admisible.

Para esto podemos hacer uso de la librería `plotly` o `bokeh`, quizás implementando un código similar al siguiente:

```python
import plotly.graph_objects as go
import numpy as np

pos_x, pos_y, espesores, sigma, sigma_admisible # Cargar datos

# Clasificar los puntos según si cumplen con la desviación estándar admisible
colors = np.where(sigma <= sigma_admisible, 'green', 'red')  # Verde si cumplen, rojo si no

# Cargar el croquis referencial como imagen
croquis_image_path = 'croquis.jpg'  # Ruta al archivo de la imagen

# Crear el scatter plot sobre el croquis
fig = go.Figure()

# Agregar la imagen de fondo (croquis)
fig.add_layout_image(
    dict(
        source=croquis_image_path,
        xref="x", yref="y",
        x=x, y=y,  # Ajustar los límites del croquis al sistema de coordenadas
        sizex=x, sizey=y,  # Ajustar el tamaño del croquis
        xanchor="left", yanchor="top",
        layer="below"
    )
)

# Agregar los puntos como scatter
fig.add_trace(go.Scatter(
    x=pos_x,
    y=pos_y,
    mode='markers',
    marker=dict(
        size=12,
        color=colors,  # Colorear según la clasificación
        line=dict(width=1, color='black')
    ),
    text=[f"Valor: {val}<br>Desv. estándar: {std}" for val, std in zip(values, std_devs)],
    hoverinfo="text"  # Mostrar el texto dinámicamente al pasar el cursor
))

plt.show()

```

Basado en lo arrojado en el gráfico anterior, sería posible remedir las zonas bajo la desviación estándar negativa admisible para corroborar la medición y tomar desiciones en base a si el valor fue correctamente medido.

Un análisis extra bajo un gráfico adaptado sería considerar los puntos sobre la desviación estándar positiva admisible, pues muchos valores en esta condición puede ser debido a mediciones incorrectas.

### Análisis de vida remanente

Finalmente, un análisis similar a la auditoría pero considerando una colorimetría respecto a un cálculo de vida remanente, permitiría una extrapolación a futuro del comportamiento de los componentes, pudiendo así alertar preventivamente a planta los componentes que preveen fallaran antes de la próxima inspección, permitiendo que ellos tomen desiciones respecto al cambio o protección adicional de los componentes en cuestión.